In [21]:
import cv2
import os
import matplotlib.pyplot as plt

image = cv2.imread('/home/charlie/projects/entropy/img/Raw_superficial/reactive/active/35_1172576-1_20170822_OCT_00002576.bmp_superficial.jpg')
#plt.imshow(image)
img_gabor = Gabor_process(image)
cv2.imwrite('/home/charlie/projects/entropy/img/test/gabor2.jpg', img_gabor)

True

<Figure size 640x480 with 0 Axes>

In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Grayscale
def BGR2GRAY(img):
    # Grayscale
    gray = 0.2126 * img[..., 2] + 0.7152 * img[..., 1] + 0.0722 * img[..., 0]
    return gray

# Gabor
def Gabor_filter(K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get half size
    d = K_size // 2

    # prepare kernel
    gabor = np.zeros((K_size, K_size), dtype=np.float32)

    # each value
    for y in range(K_size):
        for x in range(K_size):
            # distance from center
            px = x - d
            py = y - d

            # degree -> radian
            theta = angle / 180. * np.pi

            # get kernel x
            _x = np.cos(theta) * px + np.sin(theta) * py

            # get kernel y
            _y = -np.sin(theta) * px + np.cos(theta) * py

            # fill kernel
            gabor[y, x] = np.exp(-(_x**2 + Gamma**2 * _y**2) / (2 * Sigma**2)) * np.cos(2*np.pi*_x/Lambda + Psi)

    # kernel normalization
    gabor /= np.sum(np.abs(gabor))

    return gabor


def Gabor_filtering(gray, K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get shape
    H, W = gray.shape

    # padding
    gray = np.pad(gray, (K_size//2, K_size//2), 'edge')

    # prepare out image
    out = np.zeros((H, W), dtype=np.float32)

    # get gabor filter
    gabor = Gabor_filter(K_size=K_size, Sigma=Sigma, Gamma=Gamma, Lambda=Lambda, Psi=0, angle=angle)

    # filtering
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(gray[y : y + K_size, x : x + K_size] * gabor)

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

def Gabor_process(img):
    # get shape
    H, W, _ = img.shape

    # gray scale
    gray = BGR2GRAY(img).astype(np.float32)

    # define angle
    As = [0, 45, 90, 135]

    # prepare pyplot
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0.2)

    out = np.zeros([H, W], dtype=np.float32)

    # each angle
    for i, A in enumerate(As):
        # gabor filtering
        _out = Gabor_filtering(gray, K_size=111, Sigma=1.5, Gamma=1.2, Lambda=4, angle=A)
        # add gabor filtered image
        out += _out

    # scale normalization
    out = out / out.max() * 255
    #out = out.astype(np.uint8)
    kernel_size =5
    out = cv2.GaussianBlur(out,(kernel_size, kernel_size), 0)
    out = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
    out = cv2.adaptiveThreshold(out, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                cv2.THRESH_BINARY, kernel_size, 0)
    return out

img= cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/113_1296950-5_20180122_OCT_00000944.bmp_superficial.jpg')

print(img.shape)

img_gabor = Gabor_process(img)
cv2.imwrite('/home/charlie/projects/entropy/img/test/gabor5_mean.jpg', img_gabor)

(741, 741, 3)


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


<Figure size 640x480 with 0 Axes>

In [35]:
img_threshold = cv2.imread('/home/charlie/projects/entropy/raw/Raw_superficial/reactive/active/110_1296950-5_20170919_OCT_00000949.bmp_superficial.jpg')
img_threshold = cv2.cvtColor(img_threshold, cv2.COLOR_BGR2GRAY)
_, img_threshold = cv2.threshold(img_threshold,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

cv2.imwrite('/home/charlie/projects/entropy/img/test/otsu.jpg', img_threshold)
                           

True